In [1]:
%matplotlib inline

In [2]:
import sys
import numpy as np
import tensorflow as tf

from batcher import Batcher
from metrics import acc012

In [3]:
X_train = np.load('../../data/bin/train/deal.npy')
y_train = np.load('../../data/bin/train/tricks_notrump.npy')

X_val = np.load('../../data/bin/val/deal.npy')
y_val = np.load('../../data/bin/val/tricks_notrump.npy')

n_examples = X_train.shape[0]

n_h = X_train.shape[1]
n_w = X_train.shape[2]
n_c = X_train.shape[3]

batch_size = 64
n_iterations = 2000000
display_step = 10000

learning_rate = 0.001

n_hidden_units = 128

l2_reg = 0.05

In [4]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((10800000, 4, 13, 4), (100000, 4, 13, 4), (10800000, 1), (100000, 1))

In [5]:
X = tf.placeholder(tf.float32, shape=[None, n_h, n_w, n_c])
Y = tf.placeholder(tf.float32, shape=[1, None])
strides = [1,1,1,1]

conv1_w = tf.get_variable('c1w', shape=[1, 4, 4, 32], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv1_z = tf.nn.conv2d(X, filter=conv1_w, strides=strides, padding='SAME')
conv1_a = tf.nn.relu(conv1_z)

conv2_w = tf.get_variable('c2w', shape=[1, 4, 32, 64], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv2_z = tf.nn.conv2d(conv1_a, filter=conv2_w, strides=strides, padding='SAME')
conv2_a = tf.nn.relu(conv2_z)

conv3_w = tf.get_variable('c3w', shape=[1, 4, 64, 128], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv3_z = tf.nn.conv2d(conv2_a, filter=conv3_w, strides=strides, padding='SAME')
conv3_a = tf.nn.relu(conv3_z)

conv4_w = tf.get_variable('c4w', shape=[4, 4, 128, 512], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv4_z = tf.nn.conv2d(conv3_a, filter=conv4_w, strides=strides, padding='VALID')
conv4_a = tf.nn.relu(conv4_z)

fc_in = tf.contrib.layers.flatten(conv4_a)
fc_w = tf.get_variable('fcw', shape=[n_hidden_units, fc_in.shape.as_list()[1]], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
fc_b = tf.Variable(np.zeros((n_hidden_units, 1)), dtype=tf.float32)
fc_z = tf.add(tf.matmul(fc_w, tf.transpose(fc_in)), fc_b)
fc_a = tf.nn.relu(fc_z)

w_out = tf.get_variable('w_out', shape=[1, n_hidden_units], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
b_out = tf.Variable(np.zeros((1, 1)), dtype=tf.float32)
pred = tf.add(tf.matmul(w_out, fc_a), b_out)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [6]:
cost = tf.reduce_mean(tf.squared_difference(pred, Y))

In [7]:
optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9).minimize(cost)

In [8]:
init = tf.global_variables_initializer()

In [9]:
batch = Batcher(n_examples, batch_size)
cost_train_batch = Batcher(n_examples, 10000)
cost_val_batch = Batcher(100000, 10000)

In [10]:
costs = []

with tf.Session() as sess:
    sess.run(init)
    
    for iteration in range(n_iterations):
        x_batch, y_batch = batch.next_batch([X_train, y_train])
        
        if iteration % display_step == 0:
            sys.stdout.write('*')
            x_batch_c, y_batch_c = cost_train_batch.next_batch([X_train, y_train])
            x_batch_v, y_batch_v = cost_val_batch.next_batch([X_val, y_val])
            c = sess.run(cost, feed_dict={X: x_batch_c, Y: y_batch_c.T})
            costs.append(c)
            pred_train = sess.run(pred, feed_dict={X: x_batch_c, Y: y_batch_c.T})
            pred_val = sess.run(pred, feed_dict={X: x_batch_v, Y: y_batch_v.T})
            print('it={} cost={}'.format(iteration, c))
            print(acc012(y_batch_c, pred_train.T))
            print(acc012(y_batch_v, pred_val.T))
            
        sess.run(optimizer, feed_dict={X:x_batch, Y:y_batch.T})
    
    # Testing
    graph = tf.get_default_graph()
    
    X_test = np.load('../../data/bin/test/deal.npy')
    y_test = np.load('../../data/bin/test/tricks_notrump.npy')
    
    print('\n\nTesting')
    predictions = sess.run(pred, feed_dict = {X:X_test})

    print(acc012(y_test.ravel(), predictions.round().ravel()))

*it=0 cost=43.27733612060547
(0.0151, 0.054, 0.1115)
(0.0149, 0.0532, 0.1129)
*it=10000 cost=1.066941261291504
(0.4016, 0.8622, 0.9788)
(0.393, 0.865, 0.9795)
*it=20000 cost=1.0539873838424683
(0.4045, 0.8691, 0.9775)
(0.4059, 0.8659, 0.9782)
*it=30000 cost=0.8569604754447937
(0.4498, 0.8991, 0.9856)
(0.4402, 0.8991, 0.9848)
*it=40000 cost=0.7920666933059692
(0.4693, 0.9095, 0.9891)
(0.4541, 0.9138, 0.9884)
*it=50000 cost=0.839108407497406
(0.4425, 0.9052, 0.9867)
(0.459, 0.9069, 0.988)
*it=60000 cost=0.7529290318489075
(0.4773, 0.9207, 0.9885)
(0.4686, 0.921, 0.9912)
*it=70000 cost=0.7193298935890198
(0.48, 0.9286, 0.9894)
(0.4826, 0.9297, 0.9906)
*it=80000 cost=0.680478036403656
(0.4921, 0.9304, 0.9913)
(0.4965, 0.9312, 0.9902)
*it=90000 cost=0.6512296795845032
(0.5058, 0.9377, 0.9918)
(0.5084, 0.9351, 0.9917)
*it=100000 cost=0.604971706867218
(0.5255, 0.9433, 0.9935)
(0.5166, 0.9374, 0.9912)
*it=110000 cost=0.610489547252655
(0.5143, 0.9436, 0.9935)
(0.5255, 0.9429, 0.9933)
*it=1200

*it=980000 cost=0.2916974425315857
(0.6981, 0.9847, 0.9994)
(0.6839, 0.9794, 0.9976)
*it=990000 cost=0.2999739944934845
(0.6879, 0.9846, 0.9985)
(0.6724, 0.9836, 0.9988)
*it=1000000 cost=0.29510897397994995
(0.6974, 0.9855, 0.9989)
(0.6738, 0.9807, 0.9983)
*it=1010000 cost=0.29782143235206604
(0.693, 0.9842, 0.9991)
(0.6866, 0.9805, 0.998)
*it=1020000 cost=0.29463115334510803
(0.6856, 0.9858, 0.9994)
(0.6722, 0.9818, 0.9977)
*it=1030000 cost=0.29102110862731934
(0.6984, 0.985, 0.9985)
(0.6822, 0.9822, 0.9987)
*it=1040000 cost=0.2890140116214752
(0.6937, 0.9855, 0.9992)
(0.6798, 0.9794, 0.9983)
*it=1050000 cost=0.2987830340862274
(0.6925, 0.9855, 0.9987)
(0.6813, 0.9808, 0.9985)
*it=1060000 cost=0.2904503047466278
(0.6982, 0.9853, 0.9991)
(0.6889, 0.9826, 0.999)
*it=1070000 cost=0.2934224009513855
(0.6947, 0.9854, 0.9994)
(0.685, 0.9806, 0.9985)
*it=1080000 cost=0.29125699400901794
(0.7041, 0.9856, 0.9989)
(0.6849, 0.9829, 0.9981)
*it=1090000 cost=0.2977119982242584
(0.6852, 0.9847, 0.9

*it=1940000 cost=0.24674241244792938
(0.7276, 0.9912, 0.9993)
(0.6998, 0.9832, 0.9987)
*it=1950000 cost=0.24946273863315582
(0.7274, 0.9902, 0.9996)
(0.7095, 0.9854, 0.9989)
*it=1960000 cost=0.2525736689567566
(0.7219, 0.9906, 0.9993)
(0.6942, 0.9865, 0.9989)
*it=1970000 cost=0.24720990657806396
(0.7298, 0.9895, 0.9995)
(0.7006, 0.9861, 0.9988)
*it=1980000 cost=0.26606762409210205
(0.7107, 0.9894, 0.9993)
(0.7066, 0.9857, 0.9991)
*it=1990000 cost=0.2507299780845642
(0.7276, 0.9908, 0.9995)
(0.7062, 0.9866, 0.9984)


Testing
(0.70525, 0.98547, 0.99879)


In [11]:
import pandas as pd
path = '../../bridgent/preds/z_default_preds.pickle'
pd.to_pickle({'preds':predictions, 'acc012':acc012(y_test.ravel(), predictions.round().ravel())}, path)

In [12]:
pd.read_pickle(path)

{'preds': array([[5.0451465, 3.0533695, 7.033539 , ..., 5.3031974, 5.708844 ,
         8.052138 ]], dtype=float32),
 'acc012': (0.70525, 0.98547, 0.99879)}